In [3]:
import findspark
findspark.init()
import pyspark.sql as sdf
import pyspark.pandas as pds

In [4]:
spark = sdf.SparkSession.builder.appName("7amada").config("spark.sql.execution.arrow.pyspark.enabled", "true").getOrCreate()

In [5]:
spark


In [6]:
all_Data_from_Dax=pds.read_csv("El_Ma3ayeer/*.csv")

C:\Spark\spark-3.3.0-bin-hadoop3\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [9]:
all_Data_from_Dax.columns = ['_c0',
 'UniqueTaxId',
 'Name',
 'Address',
 'Period',
 'RegionId',
 'District',
 'PurchaseInOtherSalesTax',
 'SalesInOtherPurchasesTax',
 'TaxLoss',
 'Activity',
 'ActivityL2',
 'DeclarationID',
 "Dec_ID_S",
 'TaxPayerID',
 'PurchasesInOtherSales',
 'SalesInvoicesCountInOthers',
 'PurchaseInvoiceCountInOthers',
 'SalesInOtherPurchases',
 'DeclaredSalesTax']

lol = all_Data_from_Dax.to_spark()

lol.columns

['_c0',
 'UniqueTaxId',
 'Name',
 'Address',
 'Period',
 'RegionId',
 'District',
 'PurchaseInOtherSalesTax',
 'SalesInOtherPurchasesTax',
 'TaxLoss',
 'Activity',
 'ActivityL2',
 'DeclarationID',
 'Dec_ID_S',
 'TaxPayerID',
 'PurchasesInOtherSales',
 'SalesInvoicesCountInOthers',
 'PurchaseInvoiceCountInOthers',
 'SalesInOtherPurchases',
 'DeclaredSalesTax']

In [12]:
lol.createOrReplaceTempView('Frauds')
lol.show(10)

+---+-----------+--------------------+--------------------+--------+--------+--------+-----------------------+------------------------+----------+---------+----------+-------------+--------+----------+---------------------+--------------------------+----------------------------+---------------------+----------------+
|_c0|UniqueTaxId|                Name|             Address|  Period|RegionId|District|PurchaseInOtherSalesTax|SalesInOtherPurchasesTax|   TaxLoss| Activity|ActivityL2|DeclarationID|Dec_ID_S|TaxPayerID|PurchasesInOtherSales|SalesInvoicesCountInOthers|PurchaseInvoiceCountInOthers|SalesInOtherPurchases|DeclaredSalesTax|
+---+-----------+--------------------+--------------------+--------+--------+--------+-----------------------+------------------------+----------+---------+----------+-------------+--------+----------+---------------------+--------------------------+----------------------------+---------------------+----------------+
|  0|  100063721|      محسن يوسف طاهر|3 شار

In [22]:
rs = spark.sql(
#'''select UniqueTaxId , Count(*) as cnt from Frauds group by UniqueTaxId order by Count(*) desc'''
 ''' select * from (
 select row_number() over (partition by UniqueTaxId order by TaxLoss desc) as rn , * from Frauds ) as a
 where rn = 1'''
    )


In [24]:
rs.pandas_api().shape

(203146, 21)

In [25]:
spark.stop()